In [7]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from matplotlib import pyplot as plt

from PIL import Image, ImageOps
import os
import pickle

In [2]:
matlab_train_data = 'trainCharBound.mat'
matlab_test_data = 'testCharBound.mat'

train_data = loadmat(matlab_train_data)
test_data = loadmat(matlab_test_data)

In [3]:
len1 = len(train_data['trainCharBound'][0])
len2 = len(test_data['testCharBound'][0])

train_img_names = list()
train_img_lbls = list()
train_img_bb = list()

test_img_names = list()
test_img_lbls = list()
test_img_bb = list()

for i in range(len1):
    train_img_names.append(train_data['trainCharBound'][0][i][0][0])
    train_img_lbls.append(train_data['trainCharBound'][0][i][1][0])
    temp = list()
    for j in range(len(train_data['trainCharBound'][0][i][1][0])):
        temp.append(train_data['trainCharBound'][0][i][2][j])
    train_img_bb.append(temp)
    
for i in range(len2):
    test_img_names.append(test_data['testCharBound'][0][i][0][0])
    test_img_lbls.append(test_data['testCharBound'][0][i][1][0])
    temp = list()
    for j in range(len(test_data['testCharBound'][0][i][1][0])):
        temp.append(test_data['testCharBound'][0][i][2][j])
    test_img_bb.append(temp)

In [4]:
def createImageList(isTest = True):
    dir_name =  'test/' if isTest is True else  'train/'
    _, _, image_names = next(os.walk(top = dir_name))
    images = list()
    
    for name in image_names:
        image = Image.open(dir_name + name)
        image = ImageOps.grayscale(image)
        image = image.resize((100, 32))
        image = np.asarray(image, dtype = np.half)
        image /= 255
        images.append(image)
    return images

In [5]:
def rescaleBoundingBox(src, index, bb_src):
    original = src[index]
    og_image = Image.open(original)
    new_image = og_image.resize((100, 32))
    og_width, og_height = og_image.size
    new_width, new_height = new_image.size
    ratio_width = new_width / og_width
    ratio_height = new_height / og_height
    
    result = list()
    
    for i in range(len(bb_src[index])):
        og_x = bb_src[index][i][0]
        og_y = bb_src[index][i][1]
        og_width = bb_src[index][i][2] - og_x
        og_height = bb_src[index][i][3] - og_y
    
        new_x = og_x * ratio_width
        new_y = og_y * ratio_height
        new_width = bb_src[index][i][2] * ratio_width
        new_height = bb_src[index][i][3] * ratio_height
        temp = np.array([new_x, new_y, new_width, new_height], dtype = np.float32)
        result.append(temp)
    return result

In [12]:
new_test_bb = list()
new_train_bb = list()

train_images = createImageList(False)
test_images = createImageList(True)

for i, _ in enumerate(test_img_names):
    new_test_bb.append(rescaleBoundingBox(test_img_names, i, test_img_bb))

for i, _ in enumerate(train_img_names):
    new_train_bb.append(rescaleBoundingBox(train_img_names, i, train_img_bb))

In [13]:
images = train_images + test_images
image_names = train_img_names + test_img_names
image_lables = train_img_lbls + test_img_lbls
image_bbox = new_train_bb + new_test_bb

In [14]:
print(f'{len(images)}\n{len(image_names)}\n{len(image_lables)}\n{len(image_bbox)}')

5000
5000
5000
5000


In [16]:
num_samples = len(images)

for i in range(10):
    img_shard = images[(i * num_samples) // 10 : ((i + 1) * num_samples) // 10]
    label_shard = image_lables[(i * num_samples) // 10 : ((i + 1) * num_samples) // 10]
    bbox_shard = image_bbox[(i * num_samples) // 10 : ((i+1) * num_samples) // 10]
    
    with open(f'dataset/shard_{i}.pkl', 'wb') as f:
        pickle.dump((img_shard, label_shard, bbox_shard), f)